In [ ]:
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install pandas --quiet

In [79]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

filename = "../raw_data/Search results 20220531.csv"

In [80]:
read_metadata(filename)

,Title,Subtitle,CELEX number,Cellar reference,EUROVOC descriptor,ECLI identifier,Subject matter,"Subject matter, 1st level","Subject matter, 2nd level","Subject matter, 3rd level",...,"Directory code, 4th level","Directory code, 5th level","Directory code, 6th level",Date of document,Author,In force indicator,Unnamed: 21,head,body,tail
0,Regulation (EU) 2022/123 of the European Parli...,PE/76/2021/REV/1,32022R0123,197e1547-823a-11ec-8c40-01aa75ed71a1,"medical device, European Medicines Agency, inf...",NaN,"Public health, Provisions governing the Instit...","Public health, Provisions governing the Instit...",NaN,NaN,...,NaN,NaN,NaN,2022-01-25,"European Parliament, Council of the European U...",True,NaN,None,None,None
1,Regulation (EU) 2021/2303 of the European Parl...,PE/61/2021/REV/1,32021R2303,f6e0866d-6912-11ec-9136-01aa75ed71a1,"EU office or agency, operation of the Institut...",NaN,Asylum policy,Asylum policy,NaN,NaN,...,Right to asylum (application of international ...,NaN,NaN,2021-12-15,"Council of the European Union, European Parlia...",True,NaN,None,None,None
2,Regulation (EU) 2021/2282 of the European Parl...,PE/80/2021/INIT,32021R2282,177f73e7-62c9-11ec-a033-01aa75ed71a1,"public health, scientific cooperation, medical...",NaN,"Research and training, Public health","Research and training, Public health",NaN,NaN,...,NaN,NaN,NaN,2021-12-15,"European Parliament, Council of the European U...",True,NaN,None,None,None
3,Regulation (EU) 2021/2115 of the European Parl...,PE/64/2021/REV/1,32021R2115,4a220961-5636-11ec-91ac-01aa75ed71a1,"aid to agriculture, EAGF, common agricultural ...",NaN,Agricultural structural funds,Agricultural structural funds,NaN,NaN,...,NaN,NaN,NaN,2021-12-02,"Council of the European Union, European Parlia...",True,NaN,None,None,None
4,Regulation (EU) 2021/2116 of the European Parl...,PE/65/2021/INIT,32021R2116,91b120e8-5636-11ec-91ac-01aa75ed71a1,"aid to agriculture, EAGF, rural development, E...",NaN,Agricultural structural funds,Agricultural structural funds,NaN,NaN,...,NaN,NaN,NaN,2021-12-02,"Council of the European Union, European Parlia...",True,NaN,None,None,None
5,Regulation (EU) 2021/1873 of the European Parl...,PE/50/2021/REV/2,32021R1873,e321f43f-35f7-11ec-bd8e-01aa75ed71a1,"fruit-growing, industrial property, floricultu...",NaN,"Intellectual, industrial and commercial proper...","Intellectual, industrial and commercial proper...",NaN,NaN,...,NaN,NaN,NaN,2021-10-20,"European Parliament, Council of the European U...",True,NaN,None,None,None
6,Regulation (EU) 2021/1755 of the European Parl...,PE/59/2021/REV/1,32021R1755,6af609eb-27d1-11ec-bd8e-01aa75ed71a1,"economic and social cohesion, allocation of re...",NaN,"Financial provisions, Economic, social and ter...","Financial provisions, Economic, social and ter...",NaN,NaN,...,NaN,NaN,NaN,2021-10-06,"European Parliament, Council of the European U...",True,NaN,None,None,None
7,Regulation (EU) 2021/1529 of the European Parl...,PE/67/2021/INIT,32021R1529,4450df88-19ad-11ec-b4fe-01aa75ed71a1,"EU aid, political reform, EU financial instrum...",NaN,"Financial provisions, External relations, Acce...","Financial provisions, External relations, Acce...",NaN,NaN,...,NaN,NaN,NaN,2021-09-15,"Council of the European Union, European Parlia...",True,NaN,None,None,None
8,Regulation (EU) 2021/1229 of the European Parl...,PE/33/2021/REV/1,32021R1229,7d825864-f0d0-11eb-a71c-01aa75ed71a1,"green economy, public sector, climate change, ...",NaN,"Environment, Economic policy","Environment, Economic policy",NaN,NaN,...,NaN,NaN,NaN,2021-07-14,"European Parliament, Council of the European U...",True,NaN,None,None,None
9,Regulation (EU) 2021/1232 of the European Parl...,PE/38/2021/REV/1,32021R1232,23166324-f0d2-11eb-a71c-01aa75ed71a1,"personal data, social media, regulation of tel...",NaN,"Human rights, Justice and home affairs, Data p...","Human rights, Justice and home affairs, Data p...",NaN,NaN,...,NaN,NaN,NaN,2021-07-14,"Council of the European Union, European Parlia

In [81]:
def read_metadata(filename):
    data = pd.read_csv(filename)
    data['content'] = None
    return data

def get_url_and_content(cellar_ref):
    psid = cellar_ref
    psname = "cellar" # other options: cellar, celex, oj, com, genpub, ep, jurisprudence, dd, mtf, consolidation, eurostat, eesc, cor, nim, pegase, agent, uriserv, join, swd, comnat,mdr, legissum, ecli, procedure, procedure-event, eli, immc, planjo
    lancode = "0006" # language code
    doctype = "03"
    docnum = "DOC_1"
    # for further information, see Documentation Page 37: https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en/format-PDF/source-73059305
    URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"

    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    soup = BeautifulSoup(response.content, "html.parser")
    if str(soup)[1:4] == "PDF":
        '''
        in some (few) cases, the doctype is not 03 but 02. change it for these cases
        '''
        doctype = "02"
        URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
    else:
        pass
    return ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])

def append_content(filename):
    data = read_metadata(filename)
    cellar_references = data['Cellar reference']    
    for index, ref in enumerate(cellar_references):
        data['content'][index] = get_url_and_content(ref)
    return data

In [82]:
data_with_content = append_content(filename)

/tmp/ipykernel_2898/3362116268.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'][index] = get_url_and_content(ref)
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [84]:
data_with_content['content'][0]

'THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article\xa0114 and Article\xa0168(4), point (c), thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Economic and Social Committee\xa0(1), Having regard to the opinion of the Committee of the Regions\xa0(2), Acting in accordance with the ordinary legislative procedure\xa0(3), Whereas: (1) Pursuant to Articles 9 and\xa0168 of the Treaty on the Functioning of the European Union (‘TFEU’) and Article\xa035 of the Charter of Fundamental Rights of the European Union (the ‘Charter’), the Union is to ensure a high level of human health protection in the definition and implementation of all Union policies and activities. (2) The COVID-19 pandemic has highlighted the interconnectedness of human, animal, and ec

In [73]:
data_with_content = append_content(filename)
#data_with_content.to_csv("../raw_data/test_data_scraped.csv")

/tmp/ipykernel_2898/3362116268.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'][index] = get_url_and_content(ref)
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [76]:
def read_metadata(filename):
    data = pd.read_csv(filename)
    data['head'] = None
    data['body'] = None
    data['tail'] = None
    return data

def get_url_and_content(cellar_ref):
    psid = cellar_ref
    psname = "cellar" # other options: cellar, celex, oj, com, genpub, ep, jurisprudence, dd, mtf, consolidation, eurostat, eesc, cor, nim, pegase, agent, uriserv, join, swd, comnat,mdr, legissum, ecli, procedure, procedure-event, eli, immc, planjo
    lancode = "0006" # language code
    doctype = "03"
    docnum = "DOC_1"
    # for further information, see Documentation Page 37: https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en/format-PDF/source-73059305
    URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"

    response = requests.get(URL, headers={"Accept-Language":"en-US"})
    soup = BeautifulSoup(response.content, "html.parser")
    if str(soup)[1:4] == "PDF":
        '''
        in some (few) cases, the doctype is not 03 but 02. change it for these cases
        '''
        doctype = "02"
        URL = f"http://publications.europa.eu/resource/{psname}/{psid}.{lancode}.{doctype}/{docnum}"
        response = requests.get(URL, headers={"Accept-Language":"en-US"})
        soup = BeautifulSoup(response.content, "html.parser")
    else:
        pass
    content = ' '.join([item.text for item in soup.find_all("p", class_="oj-normal")])
    split_content = content.split('Whereas:', 1)
    split_content_again = split_content[1].split('HAVE ADOPTED THIS REGULATION:', 1)
    head = split_content[0]
    body = split_content_again[0]
    tail = split_content_again[1]
    return [head, body, tail]

def append_content(filename):
    data = read_metadata(filename)
    cellar_references = data['Cellar reference']    
    for index, ref in enumerate(cellar_references):
        content = get_url_and_content(ref)
        data['head'][index] = content[0]
        data['body'][index] = content[1]
        data['tail'][index] = content[2]
    return data

In [77]:
data_with_content = append_content(filename)

/tmp/ipykernel_2898/2353618802.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['head'][index] = content[0]
/tmp/ipykernel_2898/2353618802.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['body'][index] = content[1]
/tmp/ipykernel_2898/2353618802.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tail'][index] = content[2]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some

In [78]:
data_with_content

,Title,Subtitle,CELEX number,Cellar reference,EUROVOC descriptor,ECLI identifier,Subject matter,"Subject matter, 1st level","Subject matter, 2nd level","Subject matter, 3rd level",...,"Directory code, 4th level","Directory code, 5th level","Directory code, 6th level",Date of document,Author,In force indicator,Unnamed: 21,head,body,tail
0,Regulation (EU) 2022/123 of the European Parli...,PE/76/2021/REV/1,32022R0123,197e1547-823a-11ec-8c40-01aa75ed71a1,"medical device, European Medicines Agency, inf...",NaN,"Public health, Provisions governing the Instit...","Public health, Provisions governing the Instit...",NaN,NaN,...,NaN,NaN,NaN,2022-01-25,"European Parliament, Council of the European U...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) Pursuant to Articles 9 and 168 of the Tre...,Within the European Medicines Agency (the ‘Ag...
1,Regulation (EU) 2021/2303 of the European Parl...,PE/61/2021/REV/1,32021R2303,f6e0866d-6912-11ec-9136-01aa75ed71a1,"EU office or agency, operation of the Institut...",NaN,Asylum policy,Asylum policy,NaN,NaN,...,Right to asylum (application of international ...,NaN,NaN,2021-12-15,"Council of the European Union, European Parlia...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) The objective of the Union’s policy on as...,1. This Regulation establishes a European U...
2,Regulation (EU) 2021/2282 of the European Parl...,PE/80/2021/INIT,32021R2282,177f73e7-62c9-11ec-a033-01aa75ed71a1,"public health, scientific cooperation, medical...",NaN,"Research and training, Public health","Research and training, Public health",NaN,NaN,...,NaN,NaN,NaN,2021-12-15,"European Parliament, Council of the European U...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) The development of health technologies is...,1. This Regulation establishes: (a) a suppo...
3,Regulation (EU) 2021/2115 of the European Parl...,PE/64/2021/REV/1,32021R2115,4a220961-5636-11ec-91ac-01aa75ed71a1,"aid to agriculture, EAGF, common agricultural ...",NaN,Agricultural structural funds,Agricultural structural funds,NaN,NaN,...,NaN,NaN,NaN,2021-12-02,"Council of the European Union, European Parlia...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) The Commission communication of 29 Novemb...,1. This Regulation lays down rules on: (a) ...
4,Regulation (EU) 2021/2116 of the European Parl...,PE/65/2021/INIT,32021R2116,91b120e8-5636-11ec-91ac-01aa75ed71a1,"aid to agriculture, EAGF, rural development, E...",NaN,Agricultural structural funds,Agricultural structural funds,NaN,NaN,...,NaN,NaN,NaN,2021-12-02,"Council of the European Union, European Parlia...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) The Commission communication of 29 Novemb...,This Regulation lays down rules on the financ...
5,Regulation (EU) 2021/1873 of the European Parl...,PE/50/2021/REV/2,32021R1873,e321f43f-35f7-11ec-bd8e-01aa75ed71a1,"fruit-growing, industrial property, floricultu...",NaN,"Intellectual, industrial and commercial proper...","Intellectual, industrial and commercial proper...",NaN,NaN,...,NaN,NaN,NaN,2021-10-20,"European Parliament, Council of the European U...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,"(1) Technical difficulties in breeding, due t...",1. The term of the Community plant variety ...
6,Regulation (EU) 2021/1755 of the European Parl...,PE/59/2021/REV/1,32021R1755,6af609eb-27d1-11ec-bd8e-01aa75ed71a1,"economic and social cohesion, allocation of re...",NaN,"Financial provisions, Economic, social and ter...","Financial provisions, Economic, social and ter...",NaN,NaN,...,NaN,NaN,NaN,2021-10-06,"European Parliament, Council of the European U...",True,NaN,THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE...,(1) The Agreement on the withdrawal of the Un...,1. This Regulation establishes the Brexit A...
7,Regulation (EU) 2021/1529 of the European Parl...,PE/67/2021/INIT,32021R1529,4450df88-19ad-11ec-b4fe-01aa75ed71a1,"EU aid, political reform, EU financial instrum...",N